# Gridworld y su solución como MDPs

En este trabajo definiremos el ambiente de Gridworld y su solución como un MDP.
Gridworld es un ambiente clásico de prueba dentro del aprendizaje por refuerzo. Durante este taller definiremos el modelo básico del ambiente, que extenderemos incrementalmente de acuerdo a las necesidades del algoritmo de solución.

## Ambiente 🌎

El ambiente de ridworld se define como una cuadricula de `nxm`. El ambiente tiene obstaculos, es decir casillas por las cuales no puede pasar el agente. Al chocar con un obstaculo, el agente se mantiene terminaría en el mismo estado inicial. Además, el ambiente tiene una casilla de inicio, y algunas casillas de salida. Un ejemplo del ambiente para el caso `3x4` se muestra a continuación.

![gridworld.png](https://raw.githubusercontent.com/FLAGlab/isis4222-rl/a502e264157729fcb8cc00d484e4a8e8e4734a15/week3/img/gridworld.png)

En este ejemplo del ambiente el agente comienza en la casilla inferior izquierda y tiene como objetivo llegar a la casilla de salida verde, con recompensa 1. La otra casilla de salida, tiene recompensa -1.


### Task 1.
#### ¿Cómo podemos codificar el ambiente?

De una definición completa del ambiente, como una clase de python llamada `Environment`, estableciendo:
1. Un atributo que define la cuadrícula (`board`). El ambiente recibirá una matriz como parámetro describiendo la cuadrícula en el momento de su creación. Definiremos las casillas por las que puede pasar el agente como casillas vacias, las casillas por las que no puede pasar el agente con un valor none `None` y las casillas de salida con el valor asociado a la recompensa definidas para cada una de ellas.
2. Un atributo `nrows` para almacenar la cantidad de filas de la cuadrícula.
3. Un atributo `ncols` para almacenar la cantidad de columnas de la cuadrícula.
4. Un atributo `initial_state` para almacenar el estado inicial del agente dentro del ambiente.
5. Un atributo con el estado actual (`current_state`) en el que se encuentra el agente. El valor de `current_state` se definirá como una tupla 

Un ejemplo de la definición del tablero para el caso de 5x5 de la figura anterior se da a continuación.
```
board = [['', ' ', ' ',  '+1'],
         [' ', '#', ' ',  '-1'],
         ['S', ' ', ' ', ' ']]
```
En el ejemplo `S` denota el estado inicial y `'#'` la casilla prohibida (manejaremos esta convención para todos los ambientes de gridworld).

#### Comportamiento del ambeinte

Una vez definido el ambiente definimos su comportamiento. Para ello requerimos los siguientes métodos:
1. `get_current_state` que no recibe parámetros y retorna el estado actual (la casilla donde se encuentra el agente)
2. `get_posible_actions` que recibe el estado actual del agente como parámetro y retorna las acciones disponibles para dicho estado. Las acciones estarán dadas por su nombre (`'up', 'down', 'left', 'right'`). Como convención definiremos que el agente siempre puede moverse en todas las direcciones, donde un movimiento en dirección de un obstáculo o los límites del ambiente no tienen ningún efecto visible en la posición del agente.
3. `do_action` que recibe como parámetro la acción a ejecutar y retorna el valor de la recompensa y el nuevo estado del agente, como un pareja `reward, new_state`
4. `reset` que no recibe parámetros y restablece el ambiente a su estado inicial.
5. `is_terminal` que no recibe parámetros y determina si el agente está en el estado final o no. En nuestro caso, el estado final estará determinado por las casillas de salida (i.e., con un valor definido).



In [1]:
from typing import Tuple, List
from enum import Enum

import pandas as pd


class Action(Enum):
    UP = 'up'
    RIGHT = 'right'
    DOWN = 'down'
    LEFT = 'left'


State = Tuple[int, int]


class Environment:
    def __init__(self, board: List[List[str]]):
        self.board = pd.DataFrame(board)
        self.num_rows = len(board)
        self.num_cols = len(board[0])
        self.initial_state = (0, 0)
        for i, row in self.board.iterrows():
            for j, cell in enumerate(row):
                if cell == 'S':
                    self.initial_state = (j, i)
        self.current_state = self.initial_state

    def __repr__(self):
        board = self.board.copy()
        x, y = self.current_state
        board.loc[y, x] += 'C'
        return board.__repr__()

    def __str__(self):
        return self.__repr__()

    def get_current_state(self) -> State:
        return self.current_state

    @staticmethod
    def get_possible_actions(_state: Tuple[int, int]) -> List[Action]:
        return [action for action in Action]

    def get_reward(self) -> int:
        x, y = self.current_state
        try:
            return int(self.board.loc[y, x])
        except ValueError:
            return 0

    def do_action(self, action: Action) -> Tuple[int, State]:
        x, y = self.current_state
        x_, y_ = x, y
        if action == Action.UP:
            y_ = max(0, y - 1)
        elif action == Action.DOWN:
            y_ = min(self.num_rows - 1, y + 1)
        elif action == Action.LEFT:
            x_ = max(0, x - 1)
        elif action == Action.RIGHT:
            x_ = min(self.num_cols - 1, x + 1)
        if self.board.loc[y_, x_] != '#':
            self.current_state = x_, y_
        return self.get_reward(), self.current_state

    def reset(self):
        self.current_state = self.initial_state

    def is_terminal(self) -> bool:
        return self.get_reward() != 0



Teniendo en cuenta la definición del agente, genere un ambiente de `10x10` como se muestra a continuación.

![evaluacion.png](https://raw.githubusercontent.com/FLAGlab/isis4222-rl/a502e264157729fcb8cc00d484e4a8e8e4734a15/week3/img/evaluacion.png)

In [2]:
env_board = [['S'] + [' '] * 9,
             [' '] * 10,
             [' ', '#', '#', '#', '#', ' ', '#', '#', '#', ' '],
             [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' '],
             [' ', ' ', ' ', ' ', '#', '-1', ' ', ' ', ' ', ' '],
             [' ', ' ', ' ', ' ', '#', '+1', ' ', ' ', ' ', ' '],
             [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' '],
             [' ', ' ', ' ', ' ', '#', '-1', '-1', ' ', ' ', ' '],
             [' '] * 10,
             [' '] * 10
             ]

environment = Environment(env_board)

print(environment)

    0  1  2  3  4   5   6  7  8  9
0  SC                             
1                                 
2      #  #  #  #       #  #  #   
3               #                 
4               #  -1             
5               #  +1             
6               #                 
7               #  -1  -1         
8                                 
9                                 


In [3]:
print(environment.do_action(Action.DOWN))
print(environment)

(0, (0, 1))
    0  1  2  3  4   5   6  7  8  9
0   S                             
1   C                             
2      #  #  #  #       #  #  #   
3               #                 
4               #  -1             
5               #  +1             
6               #                 
7               #  -1  -1         
8                                 
9                                 



### Task 2.
Plantee el problema de MDP para cada una de las casillas. Especifique el estado de inicio, las transiciones y su probabilidad (suponiendo que todas las acciones sucede con probabilidad de 0.25) y los estados de fin con su recompensa.
¿Cómo serían las recompensas esperadas para cada estado?

In [7]:
from functools import lru_cache

S = [(x, y) for x in range(environment.num_cols) for y in range(environment.num_rows)]
A = [action for action in Action]


# P(s_ | s, a) = P(s_, s, a)

@lru_cache(maxsize=None)
def P(next_state: State, current_state: State, action: Action) -> float:
    temp_environment = Environment(environment.board)
    temp_environment.current_state = current_state
    _, environment_next_state = temp_environment.do_action(action)
    return int(environment_next_state == next_state)


@lru_cache(maxsize=None)
def R(next_state: State):
    temp_environment = Environment(environment.board)
    temp_environment.current_state = next_state
    return temp_environment.get_reward()


print(P((1, 0), (1, 1), Action.UP))
print(P((3, 0), (3, 3), Action.UP))
print(R((1, 0)))
print(R((3, 0)))

1
0
0
0



### Task 3.
Bajo la definción del problema anterior, suponga que cada acción tiene una probabilidad de éxito de 60%, con probabilidad de 30% se ejecutará la sigiente acción (en dirección de las manesillas del reloj) y con probabilidad de 10% no pasará nada. Bajo estas condiciones, ¿Cómo serían las recompensas esperadas para cada estado? 

In [5]:
@lru_cache(maxsize=None)
def P_2(next_state: State, current_state: State, action: Action) -> float:
    action_index = A.index(action)
    alternative_action = A[(action_index + 1) % len(A)]
    return 0.6 * P(next_state, current_state, action) + 0.3 * P(next_state, current_state,
                                                                alternative_action) + 0.1 * int(
        next_state == current_state)

In [6]:
s = (3, 3)
a = Action.UP

print(pd.DataFrame([
    [P_2((x, y), s, a) for x in range(environment.num_cols)] for y in range(environment.num_rows)
]))

     0    1    2    3    4    5    6    7    8    9
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0



### Task 4. 
Defina una situación de la vide real, de su escogencia, como un MDP.

### Estrategia de mercadeo

En una compañía se debe establecer una estrategia de mercadeo entre tres opciones. Cada estrategia tiene probabilidades distintas de atraer a un número particular de clientes; Sin embargo, las estrategias tienen un costo que afecta el precio del servicio dado por la compañía a los clientes que ya tiene. Mientras más alto es este precio es más probable que las personas decidan retirarse del servicio.

- Estados (S): Los estados son el número de empleados que se tienen en un momento dado.
- Acciones: las acciones posibles son las tres estrategias de mercadeo que la empresa tiene a su disposición más no ejecutar ninguna campaña.
- Probabilidades (P): `P(s'|s,a) = SUM(P(empleados_perdidos|s,precio(a))*P(empleados_ganados|a)|s'=s-empleados_perdidos+empleados_ganados)`
- Recompensa (R): Es igual a los ingresos de la empresa.